In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Lets check your GPU
if tf.config.list_logical_devices('GPU') != []:
    print("LET'S DO THISS!!! :DDDD")
else: 
    print("Oh dude... D:")

LET'S DO THISS!!! :DDDD


In [33]:
with open('data/input.txt') as f:
    content = f.read().lower()
    
print(content[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [34]:
"".join(sorted(set(content.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [35]:
text_vec_layer = keras.layers.TextVectorization(
                                                split="character",
                                                standardize="lower"
                                            )
text_vec_layer.adapt([content])
encoded = text_vec_layer([content])[0]

In [36]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), subtraindo 2 (pad e unknown)
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [37]:
n_tokens, dataset_size

(39, 1115394)

In [38]:
window_size = 128
batch_size = 32
ds = tf.data.Dataset.from_tensor_slices(encoded) # Fatia o ds em janelas de tamanho window_size
ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
ds = ds.shuffle(100_000, seed=42)
ds = ds.batch(batch_size)
ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [39]:
for X_batch, y_batch in ds.take(1):
    print(X_batch.shape, y_batch.shape)
    print(X_batch[0], y_batch[0])

(32, 128) (32, 128)
tf.Tensor(
[15  0  9  3 16 17  0  9  3  0 14  3  8  1 26 10 10 18  3 14  5  9  5 13
  7 23 10 11  3  3 24 17  0  7  5  8 17  0 15  3 13  8  0 14  3  2  6  1
  8 30 10 10 18  3  8  5  3 11  4  9 13  7 23 10  3 17 10 15  3 13  0  6
  4 25  1 17  0  5  0 24  9  3 16 17  0 22  1  2  5  2  5  3  9 27 12  0
  4 11 11  0  2  6  1  0 20  3 12  7 10 19  3  8  0 14 15  0 22  8  3  7
 22  1  8  5  2 15 30 10], shape=(128,), dtype=int64) tf.Tensor(
[ 0  9  3 16 17  0  9  3  0 14  3  8  1 26 10 10 18  3 14  5  9  5 13  7
 23 10 11  3  3 24 17  0  7  5  8 17  0 15  3 13  8  0 14  3  2  6  1  8
 30 10 10 18  3  8  5  3 11  4  9 13  7 23 10  3 17 10 15  3 13  0  6  4
 25  1 17  0  5  0 24  9  3 16 17  0 22  1  2  5  2  5  3  9 27 12  0  4
 11 11  0  2  6  1  0 20  3 12  7 10 19  3  8  0 14 15  0 22  8  3  7 22
  1  8  5  2 15 30 10 10], shape=(128,), dtype=int64)


In [40]:
# Criando função
def to_dataset(sequence, window_size=128, batch_size=32, seed=42, shuffle=False, target='all_window'):
    ds = tf.data.Dataset.from_tensor_slices(sequence) # Fatia o ds em janelas de tamanho window_size
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    if target == 'all_window':
        ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)
    elif target == 'last_char':
        ds = ds.map(lambda window: (window[:, :-1], window[:, -1])).prefetch(1)
    else:
        raise ValueError('target must be "all_window" or "last_char"')
    return ds

In [41]:
# Exemplo de uso
list(to_dataset(text_vec_layer(['To be'])[0], window_size=3))

[(<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 4,  5,  2],
         [ 5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 5,  2, 23],
         [ 2, 23,  3]], dtype=int64)>)]

In [42]:
window_size = 128
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], window_size=window_size, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], window_size=window_size)
test_set = to_dataset(encoded[1_060_000:], window_size=window_size)

### Training Char-RNN model

In [43]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [44]:
def decoder(seq, delta=2):
    output = np.array(text_vec_layer.get_vocabulary())[seq + delta]
    new_output = []
    for i in range(output.shape[0]):
        phrase = ""
        for j in range(output.shape[1]):
            phrase += output[i, j]
        new_output.append(phrase)
    return np.array(new_output)

In [45]:
# estilo
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = '#26262e' 

def print_multiple_generated(predicted_text, original_text, n=8):
    print("--> Original text:")
    for i in range(n):
        print(original_text[i])
    print("\n--> Generated text:")
    for i in range(n):
        print(predicted_text[i])
        
def plot_loss(history, step):
    plt.plot(list(range(len(history['loss']))), history['loss'], label='Training Loss')
    plt.plot(list(range(len(history['loss']))), history['val_loss'], label='Validation Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_accuracy(history, step):
    plt.plot(list(range(len(history['accuracy']))), history['accuracy'], label='Training Accuracy')
    plt.plot(list(range(len(history['accuracy']))), history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Step')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [46]:
history = {
            'text_base': 'To be or not to be, that is the question:',
            'text_base_encoded': text_vec_layer(['To be or not to be, that is the question:']),
            'list_predicted_text_base': [],
        }

class LogCallback(tf.keras.callbacks.Callback):
    def __init__(self, history):
        super().__init__()
        self.history_ = history
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"Final da época {epoch}, Loss: {logs['loss']}, Acc: {logs['accuracy']}")
    
    def on_train_batch_end(self, batch, logs=None):
        if batch % 3_000 == 0:
            print("")
            print(f"Final do lote {batch}, Loss do lote: {logs['loss']}")
            model = self.model
            predicted_text = decoder(tf.argmax(model(history['text_base_encoded'], training=False), axis=2))[0]
            print("")
            print("--> Original text: ", history['text_base'])
            print("--> Generated text: ", predicted_text)
            self.history_['list_predicted_text_base'].append(predicted_text)
            print("")

In [47]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam",
              metrics=["accuracy"])

In [48]:
history_train = model.fit(train_set, 
                    validation_data=valid_set, 
                    epochs=10,
                    callbacks=[LogCallback(history=history), 
                               model_ckpt])

Epoch 1/10

Final do lote 0, Loss do lote: 3.6630570888519287

--> Original text:  To be or not to be, that is the question:
--> Generated text:  e   n            nf  e   n   ee      d   

   1408/Unknown - 35s 16ms/step - loss: 2.0736 - accuracy: 0.3985

KeyboardInterrupt: 

In [115]:
y_proba_ = model(X_batch)
y_pred_ = tf.argmax(y_proba_, axis=2)  # choose the most probable character ID
predicted_text = decoder(y_pred_)
original_text = decoder(X_batch)
predicted_text, original_text

(array(['i e t poman  tor y cannot \nte ng o melsce,\nae nhet i am \nconsition.\n\nhat io d momfition oap tssraat .sord tn the pert thet h  t ',
        'eo  d tirrs ahe corinint hnes the gomntenlowdgiart  wha cnm tfr btldier, nur seaed toe coat\nwhe crwguesmfr rritpetsr.\n\nhth auher',
        'e nn:aomber,\ntheu h thet s th t l  tyst i dheedt trom tul  tha cest whall be t the cutiness,in tome tther trrht \nwn chrse till h',
        'en and ti lome  ioneral  tnd toare wallome.onl \n\nmenenius:\nw gamgred iheu and tilcome ,\ni aauld ni p,t d tnsauld birgh  t am hog',
        "uehres to tine yss restrnetion, ai pe maght we canl'd your voltuisher.\n\ncoriolanus:\nwy an ium tifes hea\n\ncartius:\nt  antium \n\nco",
        's:\nt ll nnep youraomeany \nietl you arlne \n\ncrutus:\nwh hhan tere tor the people,\n\ncicinius:\ntore you tell.\nhe has at iot, snd te ',
        'tthe common  tou sholl bond yotmutlichaenemit teach tou tepeive\ntut tt iroveeds tf womis toom the  to tourtnd to miy so

In [ ]:
# y_proba_ = model(X_batch)
# y_pred_ = tf.argmax(y_proba_, axis=-1)
# decoder(y_pred_)[:10]

array(['e  nn o iie   ai    inn  ee   e o oe i e        o nnee  n   ieeiee        e o     nnne   ieeieiee  e  ie i ne iiie ii  een  ieie',
       'oee a  oo ee   i ii  i  e e i inne nnno nnin i e i i    e   iiiei e   e  a a nee ieeee   nnnnn  iin i iie enee i iieei  ni i iei',
       'ei  nneeeee  niiii nnn i ii n  ie ii   e i iiii i   i   ea i  oi i   e aoa   ioeeoe     i eie   i   e  i i  ie ani ianeineiniei ',
       'ieeiiii  i i e ne o i a o  eeeeienineeeeneeeee  nnn ieeie       ee e   in i iee ie iiinneon         a e nn in iie  ieeeeeee i  e',
       'iinn  iiie     ee   eaa i i i  i i  i  n oi i eae ii n iieieee    o   i  ineeeiiieeie a     ie ieeio nnen e   n  o   e o eeeee o',
       'aeea    e   nnnn ie a  e o   iei i neiee  i i i  e e   ieii      ioonnn io   i n ni i  e  neeie a  oo ee   i ii  i  e e i inne n',
       'i ii  ieei iii  eo   e i  nen nninn   e  eeei n i  ieee i  i  e oeo e  i  iiii e   n       n i   aa i  i  eii  eniii     ii ee e',
       'oe i ii iieiee o e 

In [ ]:

# y_pred_ = tf.argmax(y_proba_)  # choose the most probable character ID
# len(decoder(y_pred_)[0])